In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html

import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from CRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "User"
database = "AAC"
shelter = AnimalShelter(username, password, database)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #adds link to image
    html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))],
          href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('Garrett Waag SNHU CS-340 Dashboard'))),
    html.Hr(),
    #data filter buttons for datatable
    html.Div(className='row',
            style={'display' : 'flex'},
                children=[
                    dcc.RadioItems(id='radio-items',
                        options=[
                        {'label': 'Water', 'value': 'Water'},
                        {'label': 'Mountain/Wilderness', 'value': 'Wild'},
                        {'label': 'Disaster/Individual', 'value': 'Indi'},
                        {'label': 'Reset', 'value': 'Reset'}
                        ],
                        value='Reset'
                    )
                ]),
    html.Hr(),
    
    #sets datatable to only select 1 row and can sort by any column
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_current= 0,
        page_size= 10, 
        row_selectable='single',
        selected_columns=[],
        selected_rows=[0],
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id = 'graph-id'),
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-items', 'value')])

#sets filters for db
def update_dashboard(value):
    
    if (value == 'Reset'):
        df = pd.DataFrame(shelter.read({}))
    
    elif (value == 'Water'):
        df = pd.DataFrame(shelter.read(
            {'breed':{'$in':['Labrador Retriever Mix','Chesapeake Bay Retriever','Newfoundland']},
            'sex_upon_outcome':'Intact Female','age_upon_outcome_in_weeks':{'$gte':26},
             'age_upon_outcome_in_weeks':{'$lte':156}}))
    elif(value == 'Wild'):
        df = pd.DataFrame(shelter.read(
            {'breed':{'$in':['German Shepherd','Alaskan Malamute','Old English Sheepdog','Siberian Husky','Rottweiler']},
            'sex_upon_outcome':'Intact Male','age_upon_outcome_in_weeks':{'$gte':26},
             'age_upon_outcome_in_weeks':{'$lte':156}}))
        
    elif(value == 'Indi'):
        df = pd.DataFrame(shelter.read(
            {'breed':{'$in':['German Shepherd','Doberman Pinscher','Golden Retriever','Bloodhound','Rottweiler']},
            'sex_upon_outcome':'Intact Male','age_upon_outcome_in_weeks':{'$gte':20},
             'age_upon_outcome_in_weeks':{'$lte':300}}))
    
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(Output('datatable-id', 'style_data_conditional'),
            [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#populates graph
@app.callback(Output('graph-id', "figure"),
            [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    pie_df = pd.DataFrame.from_dict(viewData)
    pie = px.pie(pie_df, names = 'breed', title ='Breeds')
    return pie
            
    
#populates map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")])
def update_map(viewData, rows):
    
    
    dff = pd.DataFrame.from_dict(viewData)
    
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[rows[0],'location_lat'],dff.loc[rows[0],'location_long']], children=[
                dl.Tooltip(dff.loc[rows[0],'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[rows[0],'name'])
                ])
            ])
        ])
    ]
app